In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install datasets>=1.18.3
!pip install transformers
!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f
#==4.11.3
!pip install librosa
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 80.4 MB/s eta 0:00:00

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

-f option requires 1 argument
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 67.4 MB/s eta 0:00:00


In [3]:
# from huggingface_hub import notebook_login

# notebook_login()

In [4]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [5]:
# Creating a dataframe with info about the dataset
import pandas as pd
import os

limit = 14

# For positive reviews
df = pd.read_csv('/content/drive/MyDrive/Navana Internship/15_shortest_positive.csv')
root = '/content/drive/MyDrive/Navana Internship'
path = []
for i, _ in df.iterrows():
    path.append(root + '/Audio/P-' + str(i+1) + '.m4a')
    
    if i == limit:
        break

df = df[df.index < limit+1]

df['path'] = path
df.to_csv(os.path.join(root, 'Dataset', 'positive.csv'))

# For negative reviews
df = pd.read_csv('/content/drive/MyDrive/Navana Internship/15_shortest_negative.csv')
root = '/content/drive/MyDrive/Navana Internship'
path = []
for i, _ in df.iterrows():
    path.append(root + '/Audio/N-' + str(i+1) + '.m4a')
    
    if i == limit:
        break
df = df[df.index < limit+1]

df['path'] = path
df.to_csv(os.path.join(root, 'Dataset', 'negative.csv'))

In [6]:
# Converting to the required format

def dataset():

    train_data = []
    labels = []

    df = pd.read_csv('/content/drive/MyDrive/Navana Internship/Dataset/positive.csv')

    for i, row in df.iterrows():
        train_data.append({'path': row['path'], 'text': row['Reviews']})
        labels.append(1)
    
    df = pd.read_csv('/content/drive/MyDrive/Navana Internship/Dataset/negative.csv')

    for i, row in df.iterrows():
        train_data.append({'path': row['path'], 'text': row['Reviews']})
        labels.append(0)

    return train_data, labels

train_set, labels = dataset()

In [7]:
df.head()

,Unnamed: 0,Reviews,path
0,0,"!!! Spoiler alert!!!The point is, though, that...",/content/drive/MyDrive/Navana Internship/Audio...
1,1,!!!! MILD SPOILERS !!!!The premise goes like t...,/content/drive/MyDrive/Navana Internship/Audio...
2,2,!!!! MILD SPOILERS !!!!With the exception of T...,/content/drive/MyDrive/Navana Internship/Audio...
3,3,!!!! POSSIBLE MILD SPOILER !!!!!As I watched t...,/content/drive/MyDrive/Navana Internship/Audio...
4,4,!!!!! OF COURSE THERE'S SPOILERS !!!!! I'm sur...,/content/drive/MyDrive/Navana Internship/Audio...


In [8]:
labels

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [9]:
"""
Preparing the vocab and tokenizer
"""

for i, row in df.iterrows():
    all_text = " ".join(row["Reviews"])

vocab = list(set(all_text))
# vocab = {"vocab": [vocab], "all_text": [all_text]}

vocab_dict = {v: k for k, v in enumerate(vocab)}
vocab_dict

# Replace " " with "|"
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]


# Adding CTC blank token
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

# Saving vocab as json file
import json
with open(os.path.join('/content/drive/MyDrive/Navana Internship/vocab', 'vocab.json'), 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

vocab_dict, len(vocab_dict)

({'e': 0,
  'B': 1,
  't': 2,
  'i': 3,
  'x': 4,
  'c': 5,
  'H': 6,
  'C': 7,
  'y': 8,
  'b': 9,
  's': 10,
  'W': 11,
  'T': 12,
  'r': 13,
  'm': 15,
  '(': 16,
  ',': 17,
  '/': 18,
  'E': 19,
  'p': 20,
  'a': 21,
  'z': 22,
  'n': 23,
  'I': 24,
  'l': 25,
  'S': 26,
  'u': 27,
  'M': 28,
  'A': 29,
  '.': 30,
  ')': 31,
  '-': 32,
  'o': 33,
  '"': 34,
  'k': 35,
  'N': 36,
  'g': 37,
  'v': 38,
  'd': 39,
  'h': 40,
  'f': 41,
  "'": 42,
  'w': 43,
  '|': 14,
  '[UNK]': 44,
  '[PAD]': 45},
 46)

In [10]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer(os.path.join('/content/drive/MyDrive/Navana Internship/vocab', 'vocab.json'), unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [11]:
# repo_name = "gokul-g-menon/wav2vec2-internship"
# tokenizer.push_to_hub(repo_name)

In [12]:
import torchaudio
def speech_file_to_array_fn(batch):

  item = {}
  item['audio'] = {}

  item['audio']['path'] = batch['path']

  speech_array, sampling_rate = torchaudio.load(batch["path"])

  item['audio']["array"] = speech_array[0]
  item['audio']['sampling_rate'] = sampling_rate

  item['sentence'] = batch['text']
  
  return item

# Loading the audio
for i, items in enumerate(train_set):
    train_set[i] = speech_file_to_array_fn(items)


In [13]:
train_set[0]['audio']['sampling_rate']

32000

In [14]:
# Resampling the audio for Wav2Vec2

from datasets import Audio
import torchaudio
import librosa
import numpy as np

def resample(batch):
    speech_array, sampling_rate = torchaudio.load(str(batch['audio']["path"]))

    if sampling_rate != 16000:
      batch['audio']["array"] = np.asarray(librosa.resample(np.asarray(speech_array[0]), sampling_rate, 16_000))  
    
    batch['audio']["sampling_rate"] = 16_000

    return batch

# Loading the audio
for i, items in enumerate(train_set):
    train_set[i] = resample(items)


In [15]:
train_set[0]['audio']['sampling_rate']

16000

In [16]:
import IPython.display as ipd
import numpy as np
import random

# Play random audio
rand_int = random.randint(0, len(train_set))

print(train_set[rand_int]["sentence"])
ipd.Audio(data=np.asarray(train_set[rand_int]["audio"]["array"]), autoplay=True, rate=16000)

"A Guy Thing" tries to capture the feeling of "There's Something About Mary" or "Meet the Parents" but comes off more like it was edited up out of cutting-room rejects of those two films. Thankfully I rented it on a 5-day rental because I couldn't sit and watch more than 20 minutes at a time.The premise is decent and I liked the scenes where other guys automatically cover up for Paul's missteps (the checker at the Save-mart was great) but the script-writing is absolutely horrible.


In [17]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [18]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [19]:
preprocessed_text = tokenizer(train_set[0]['sentence'])

In [20]:
processor(train_set[0]['audio']["array"], sampling_rate=train_set[0]['audio']["sampling_rate"])#.input_values[0]

{'input_values': [array([ 1.7407334e-05,  1.7407334e-05,  1.7407334e-05, ...,
       -1.2410759e-01, -1.1034810e-01, -6.9280826e-02], dtype=float32)]}

In [21]:
def prepare_dataset(batch):

    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

# Obtaining the tensors
for i, items in enumerate(train_set):
    print(i)

    # if i != 0:
    train_set[i] = prepare_dataset(items)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [22]:
"""
Audio Pre-processing done
"""

train_set[0].keys(), train_set[0]['audio'].keys()

(dict_keys(['audio', 'sentence', 'input_values', 'labels']),
 dict_keys(['path', 'array', 'sampling_rate']))

In [23]:
"""
The dataset has been preprocessed and loaded in the variable train_set

    - train_set[0]['input_values']    -   Tensor input to the model for the first audio
    - train_set[0]['labels']    -   The output labels for each audio that map into the vocab

"""

"\nThe dataset has been preprocessed and loaded in the variable train_set\n\n    - train_set[0]['input_values']    -   Tensor input to the model for the first audio\n    - train_set[0]['labels']    -   The output labels for each audio that map into the vocab\n\n"

In [24]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [25]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [26]:
"""
Breaking the model into 3 encoder layers
"""

from transformers import Wav2Vec2Model, Wav2Vec2Config

configuration = Wav2Vec2Config(num_hidden_layers=1)
model = Wav2Vec2Model(configuration)

configuration = model.config

configuration

Wav2Vec2Config {
  "activation_dropout": 0.1,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_norm": "group",
  "feat_proj_dropout": 0.0,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-

In [27]:
model

Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (2): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (3): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (4): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5): Wav2Vec2NoLay

In [28]:
train_set[0].keys()

dict_keys(['audio', 'sentence', 'input_values', 'labels'])

In [29]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cpu").unsqueeze(0)

    return model(input_values)

results = []
# Obtaining the tensors
for i, items in enumerate(train_set):

    results.append(list(map_to_result(items).last_hidden_state[0]))

print(results)

# Saving the results to a dataframe
data = {'Sentence': [], 'Tensor':[]}

for i, output in enumerate(results):
    data['Sentence'].append(train_set[i]['sentence'])
    data['Tensor'].append(output[0])
data

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'Sentence': ["A Turkish Bath sequence in a film noir located in New York in the 50's, that must be a hint at something ! Something that curiously, in all the previous comments, no one has pointed out , but seems to me essential to the understanding of this movie the Turkish Baths sequence: a back street at night, the entrance of a sleazy sauna, and Scalise wrapped in a sheet, getting his thighs massaged.",
  " I suppose this is not the best film ever made but I voted it at 10 stars all the same. Mainly because of my feelings at the end. I and all the people around me were simply touched. This is something you don't often feel .",
  " It's common knowledge and has been said before: No one can ever play Scarlett and Rhett like Vivien Leigh and Clark Gable. Joanne Whalley Kilmer (no longer Kilmer having been divorced from ex-hubby Val Kilmer) plays her own Scarlett and although this is a sequel and not a re-make (God-forbid!!!) she still cannot rise to the occasion (i.e. her voice sounds

In [30]:
# Create DataFrame
df = pd.DataFrame(data)
"""
Saving the intermediate tensors
"""

df.to_csv(os.path.join(root, 'Results', 'Inter-tensors.csv'))

In [31]:
"""
Freezing all  the layers of the broken model
"""

model.freeze_feature_extractor()

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1213: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [32]:
model

Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (2): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (3): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (4): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5): Wav2Vec2NoLay

In [33]:
#defining dataset class
import torch
from torch.utils.data import Dataset, DataLoader
class dataset(Dataset):
  def __init__(self,x,y):
    self.x = x
    self.y = torch.tensor(y,dtype=torch.float32)
    self.length = len(self.x)
 
  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
  def __len__(self):
    return self.length

trainset = dataset(train_set, labels)

#DataLoader
trainloader = DataLoader(trainset,batch_size=1,shuffle=False)

In [34]:
"""
Building the binary classifier on top of the broken model and training it with the 
dataset to obtain the results
"""
from torch import nn

class CustomModel(nn.Module):

    def Model(self, batch):
        
        with torch.no_grad():
            input_values = torch.tensor(batch["input_values"], device="cpu").unsqueeze(0)
        
        return model(input_values[0]).last_hidden_state

    def __init__(self):#,checkpoint,num_labels): 
        super(CustomModel,self).__init__() 

        #Load Model with given checkpoint and extract its body
        self.dropout = nn.Dropout(0.1) 
        self.layer = nn.Linear(768, 1) # load and initialize weights

    def forward(self, x_train):
        
        #Extract outputs from the body
        outputs = self.Model(x_train)

        #Add custom layers
        sequence_output = self.dropout(outputs) #outputs[0]=last hidden state
        out_inter = self.layer(sequence_output)

        out = torch.sigmoid(nn.Linear(len(out_inter[0]), 1)(torch.t(out_inter[0])))

        return out

In [37]:
#forward loop
losses = []
accur = []
epochs = 8
learning_rate = 0.01
custom_model = CustomModel()#input_shape=x.shape[1])
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.SGD(custom_model.parameters(),lr=learning_rate)


for i in range(epochs):

    for j,(x_train,y_train) in enumerate(trainloader):

        #calculate output
        output = custom_model(x_train)

        #calculate loss
        loss = loss_fn(output,y_train.reshape(-1,1))

        #accuracy
        # predicted = model(torch.tensor(x,dtype=torch.float32))
        # acc = (predicted.reshape(-1).detach().numpy().round() == y).mean()

        #backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    if i%2 == 0:
        losses.append(loss)
        # accur.append(acc)
        print("epoch {}\tloss : {}\t".format(i,loss))

"""
Printing the training results
"""

<ipython-input-34-38124b545150>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_values = torch.tensor(batch["input_values"], device="cpu").unsqueeze(0)


epoch 0	loss : 0.5022517442703247	
epoch 2	loss : 1.3879806995391846	
epoch 4	loss : 0.4672163128852844	
epoch 6	loss : 0.8236573934555054	


'\nPrinting the training results\n'